In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv("data/US_Accidents_March23.csv")
data.shape

(7728394, 46)

In this notebook, we load and process The US Traffic Accident dataset for analysis and modeling. We clean the data and also extract some potentially useful information.

## 1. Dataset Overview

The US Traffic Accident dataset from [Kaggle](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents) provides comprehensive information on traffic accidents across the United States from 2016 to 2023. It contains 7,728,394 rows and 24 columns, each representing a different attribute of the accidents. Here's a brief overview of some of the key columns:

* `ID`: Unique identifier for each accident.
* `Source`: Source of the accident report (e.g., 911 call, news).
* `Severity`: Accident severity rating (on a scale from 1 to 4).
* `Start_Time`: Start time of the accident.
* `End_Time`: End time of the accident.
* `Start_Lat/Start_Lng`: Latitude and longitude where the accident started.
* `End_Lat/End_Lng`: Latitude and longitude where the accident ended (many missing values).
* `Distance(mi)`: Distance covered by the accident.
* `Description`: Brief description of the accident.
* `Street`, `City`, `County`, `State`, `Zipcode`: Location details of the accident.
* `Country`: Country where the accident occurred (all should be the USA).
* `Timezone`: Timezone of the accident location.
* `Airport_Code`: Nearest airport to the accident location.
* `Weather_Timestamp`: Time when weather data was recorded.
* `Temperature(F)`, `Wind_Chill(F)`, `Humidity(%)`, `Pressure(in)`, `Visibility(mi)`, `Wind_Direction`, `Wind_Speed(mph)`, `Precipitation(in)`, `Weather_Condition`: Various weather-related attributes.
* `Amenity`, `Bump`, `Crossing`, `Give_Way`, `Junction`, `No_Exit`, `Railway`, `Roundabout`, `Station`, `Stop`, `Traffic_Calming`, `Traffic_Signal`, `Turning_Loop`: Boolean indicators for the presence of specific road features.
* `Sunrise_Sunset`, `Civil_Twilight`, `Nautical_Twilight`, `Astronomical_Twilight`: Time of day indicators related to the position of the sun.

## 2. Data Processing

### 2.2. Handling Missing Values

In [3]:
init_nrows = data.shape[0]

In [4]:
missing_values = data.isnull().sum()
missing_values = missing_values[missing_values > 0]

Percentage of missing values per column

In [5]:
#missing_values

In [6]:
100 * missing_values / data.shape[0]

End_Lat                  44.029355
End_Lng                  44.029355
Description               0.000065
Street                    0.140637
City                      0.003274
Zipcode                   0.024779
Timezone                  0.101030
Airport_Code              0.292881
Weather_Timestamp         1.555666
Temperature(F)            2.120143
Wind_Chill(F)            25.865904
Humidity(%)               2.253301
Pressure(in)              1.820288
Visibility(mi)            2.291524
Wind_Direction            2.267043
Wind_Speed(mph)           7.391355
Precipitation(in)        28.512858
Weather_Condition         2.244438
Sunrise_Sunset            0.300787
Civil_Twilight            0.300787
Nautical_Twilight         0.300787
Astronomical_Twilight     0.300787
dtype: float64

As we want to predict accident severity, some columns may be redundant or not useful for our predictive modeling.
* `ID`: Unique identifier, not useful for prediction.
* `Source`: Source of the accident report; may not impact severity directly.
* `End_Lat/End_Lng`: High percentage of missing values and likely less useful.
* `Description`: Free-text description; challenging to use directly without NLP techniques.
* `Country`: All entries should be the USA, providing no variance.
* `Airport_Code`: Specific to the nearest airport, which may not directly impact severity.
* `Civil_Twilight`, `Nautical_Twilight`, `Astronomical_Twilight`: They could be less useful or redundant given the information provided by `Sunrise_Sunset`.
* `Weather_Timestamp`: May be redundant but useful for verification and imputation of weather information.
* `Wind_Chill(F)`: Given that `Wind_Chill(F)` is derived from `Temperature(F)` and `Wind_Speed(mph)`, it may be redundant.
* `Street`: Steet names may not be useful on their own but we could use them to determine the type of road (street, highway, etc.)
* `Timezone`: Given we have the local time and day/night status (`Sunrise_Sunset`), the time zone may not be relevant.
* `Wind_Direction`: Compared to other weather information, `Wind_Direction` nay not have a significant importance.

In [7]:
df_clean = data.drop(columns=[
    "Source", "End_Lat", "End_Lng", "Description", "Country", "Airport_Code", "Civil_Twilight", "Wind_Direction",
    "Nautical_Twilight", "Astronomical_Twilight", "Wind_Chill(F)", "Timezone", "Weather_Timestamp", "Zipcode"]).copy()

In [8]:
del(data)

After removing redundant or not so useful columns, we still have a few with missing values. Most columns are missing a very small percentage of values, except for `Precipitation(in)` which is missing 28.5\%.

#### Fixing DateTime datatypes and Splitting Date and Time

In [9]:
#df_clean["Weather_Timestamp"] = pd.to_datetime(df_clean["Weather_Timestamp"])
df_clean["Start_Time"] = pd.to_datetime(df_clean["Start_Time"])
df_clean["End_Time"] = pd.to_datetime(df_clean["End_Time"])

In [10]:
df_clean["Date"] = df_clean["Start_Time"].dt.date
df_clean["Start_Time"] = df_clean["Start_Time"].dt.time
df_clean["End_Time"] = df_clean["End_Time"].dt.time

#### Dropping rows with missing values

We have 336307 unique street names and 10869 (0.14\%) of data points have missing values. Given that it is a very small percentage, we could remove those rows.

In [11]:
df_clean = df_clean[~df_clean["Street"].isnull()]

Some city names are missing (253 entries). This is an incredibly small amount (~0.003\%). We could also drop these rows.

In [12]:
df_clean = df_clean[~df_clean["City"].isnull()]

Similar to `Street` and `City`, a small percentage of data points (0.024\%) are missing `Zipcode`. `Zipcode` provide more granular information than `City` and `County` so a correlation analysis would be needed to determine if we keep the column or not. However, given the small number of missing values, we could drop these rows.

`Temperature(F)` is a fairly useful information about weather. We could use location (`City`, `County`, etc.) and time information (`Weather_Timestamp`) to fill the missing values. However, since very few data points are missing `Temperature(F)`, we could remove those entries. A similar thing goes for other weather data.

In [14]:
df_clean = df_clean[~df_clean["Temperature(F)"].isnull()]

In [15]:
df_clean = df_clean[~df_clean["Visibility(mi)"].isnull()]

In [16]:
df_clean = df_clean[~df_clean["Pressure(in)"].isnull()]

In [17]:
df_clean = df_clean[~df_clean["Humidity(%)"].isnull()]

In [18]:
df_clean = df_clean[~df_clean["Weather_Condition"].isnull()]

~28\% of the data is missing values for `Precipitation(in)`. This is a significant amount so we could use imputation to fill the missing values. For instance, missing precipiation could mean there were no rain. To validate this, we could use `Weather_Condition`.

Let's extract `Weather_Category` and `Weather_Intensity` information.

In [19]:
def extract_weather_intensity(condition):
    intensity_terms = "(light|heavy|patches|small|widspread|partial)"
    match = re.search(intensity_terms, condition.lower())
    if match:
        return match.group(1)
    return "regular"

In [20]:
def extract_weather_category(condition):
    regular_weather = "(cloud|overcast|rain|drizzle|thunderstorm|thunder|t-storm|tornado|snow|haze|fog|mist|smoke|sand|dust|hail|squalls|ice pellets|sleet|wintry mix)"
    special_weather = "(fair|clear|volcanic ash|fair windy|showers in the vicinity|n/a precipitation|fair)"
    match = re.search(special_weather, condition.lower())
    if match:
        return match.group(1)
    match = re.search(regular_weather, condition.lower())
    if match:
        return match.group(1)
    return "other"

In [21]:
weather_categories = {
    "cloud": "cloudy",
    "overcast": "cloudy",
    "fair": "clear condition",
    "clear": "clear condition",
    "rain": "precipitation",
    "drizzle": "precipitation",
    "showers in the vicinity": "precipitation",
    "t-storm": "thunderstorm",
    "thunder": "thunderstorm",
    "thunderstorm": "thunderstorm",
    "wintry mix": "precipitation",
    "n/a precipitation": "precipitation",
    "mist": "precipitation",
    "sleet": "precipitation",
    "ice pellets": "precipitation",
    "hail": "precipitation",
    "snow": "snowstorm",
    "fog": "visibility issue",
    "haze": "visibility issue",
    "smoke": "visibility issue",
    "dust": "visibility issue",
    "sand": "visibility issue",
    "volcanic ash": "visibility issue",
    "tornado": "extreme condition",
    "squalls": "extreme condition",
}

In [22]:
%%time
df_clean["Weather_Category"] = df_clean["Weather_Condition"].apply(extract_weather_category)
df_clean["Weather_Category"] = df_clean["Weather_Category"].replace(weather_categories)

CPU times: total: 18.1 s
Wall time: 19.7 s


In [23]:
df_clean["Weather_Category"].value_counts()

clear condition      3368809
cloudy               3138013
precipitation         544424
visibility issue      196622
snowstorm             159410
thunderstorm           73965
extreme condition        100
Name: Weather_Category, dtype: int64

In [24]:
%%time
df_clean["Weather_Intensity"] = df_clean["Weather_Condition"].apply(extract_weather_intensity)
df_clean["Weather_Intensity"] = df_clean["Weather_Intensity"].replace({"patches": "light", "patches": "light", "small": "light", "partial": "light"})

CPU times: total: 6.23 s
Wall time: 6.88 s


In [25]:
df_clean["Weather_Intensity"].value_counts()

regular    6884176
light       544411
heavy        52756
Name: Weather_Intensity, dtype: int64

We set data points with missing `Precipitation(in)` where the `Weather_Category` isn't a precipitation to 0.

In [26]:
df_clean["Precipitation(in)"].describe()

count    5.440793e+06
mean     8.005682e-03
std      9.134816e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.647000e+01
Name: Precipitation(in), dtype: float64

In [27]:
df_clean.loc[(df_clean["Precipitation(in)"].isnull()) & (df_clean["Weather_Category"].str not in {"precipitation", "snowstorm", "thunderstorm"}), "Precipitation(in)"] = 0

About 7\% of the data is missing `Wind_Speed(mph)` and 2\% is missing `Wind_Direction`.

In [28]:
%%time
for category in df_clean["Weather_Category"].unique():
    median = df_clean.loc[df_clean["Weather_Category"] == category, "Wind_Speed(mph)"].median()
    df_clean.loc[df_clean["Weather_Category"] == category, "Wind_Speed(mph)"] = df_clean.loc[df_clean["Weather_Category"] == category, "Wind_Speed(mph)"].fillna(median)

CPU times: total: 6.53 s
Wall time: 7.08 s


In [29]:
df_clean["Wind_Speed(mph)"].isnull().sum()

0

Some rows in our dataset are missing day/time information in `Sunrise_Sunset`. We could use manually fill the missing values using the `Time` data. However, given that very few rows are missing this information (~0.3%), we could just drop them.

In [30]:
df_clean = df_clean[~df_clean["Sunrise_Sunset"].isnull()]

In [31]:
missing_values = df_clean.isnull().sum()
missing_values[missing_values > 0].shape[0]

0

In [32]:
print(f"{100 * (init_nrows - df_clean.shape[0]) / init_nrows:.2f}% of the original dataset was dropped")

3.45% of the original dataset was dropped


### 2.2. Fixing Inconsistencies 

#### Numerical Variables
The values for some of the numerical variables are unrealistic, namely `Temperature(F)`, `Distance(mi)`, `Pressure(in)`, `Visibility(mi)`, and `Wind_Speed(mph)`.
For instance, the highest recorded temperature on Earth is around 134°F so a max of 203 is unrealistic. Similarly, the highest wind speeds recorded in hurricanes and tornadoes are well below 1087 mph.

In [33]:
df_clean.describe()

,Severity,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06,7.461674e+06
mean,2.211411e+00,3.618952e+01,-9.473090e+01,5.547669e-01,6.170302e+01,6.482619e+01,2.954106e+01,9.092840e+00,7.646129e+00,5.817365e-03
std,4.857197e-01,5.094364e+00,1.736749e+01,1.757849e+00,1.899068e+01,2.281596e+01,1.000555e+00,2.677298e+00,5.271436e+00,7.803928e-02
min,1.000000e+00,2.455480e+01,-1.246238e+02,0.000000e+00,-4.500000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.338562e+01,-1.172264e+02,0.000000e+00,4.900000e+01,4.800000e+01,2.937000e+01,1.000000e+01,4.600000e+00,0.000000e+00
50%,2.000000e+00,3.579481e+01,-8.781335e+01,2.700000e-02,6.400000e+01,6.700000e+01,2.986000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,4.010201e+01,-8.038220e+01,4.560000e-01,7.600000e+01,8.400000e+01,3.003000e+01,1.000000e+01,1.000000e+01,0.000000e+00
max,4.000000e+00,4.900220e+01,-6.711317e+01,4.417500e+02,2.030000e+02,1.000000e+02,5.863000e+01,1.400000e+02,1.087000e+03,3.647000e+01


We can discard some inomalies and outliers for the numerical variables based on the following real-world observations:
* Temperatures as low as -60 °F can be observed in extreme cold regions, while temperatures up to 130 °F can be seen in extreme hot regions.
* Most accidents occur over short distances. A maximum of 50 miles accounts for some exceptional cases.
* Normal atmospheric pressure at sea level ranges from about 27 to 31 inches of mercury.
* Visibility can drop to 0 in dense fog, and clear conditions might extend visibility to about 20 miles.
* Typical wind speeds can vary from calm (0 mph) to extreme conditions like hurricanes and tornadoes, which can reach up to 150 mph.

In [34]:
print(f'{100 * (df_clean["Distance(mi)"] > 10).sum() / df_clean.shape[0]:.2f}% of accidents occured over 10 miles or more')
print(f'{100 * (df_clean["Distance(mi)"] > 50).sum() / df_clean.shape[0]:.2f}% of accidents occured over 50 miles or more')

0.45% of accidents occured over 10 miles or more
0.01% of accidents occured over 50 miles or more


In [35]:
df_clean = df_clean[df_clean["Distance(mi)"] <= 10]

In [36]:
print(f'{100 * (df_clean["Temperature(F)"] > 130).sum() / df_clean.shape[0]:.5f}% of the data reports a temperature of over 130F')

0.00053% of the data reports a temperature of over 130F


In [37]:
df_clean = df_clean[df_clean["Temperature(F)"] <= 130]

In [38]:
print(f'{100 * ((df_clean["Pressure(in)"] < 25) | (df_clean["Pressure(in)"] > 31)).sum() / df_clean.shape[0]:.2f}% of the data reports an atmospheric pressure outside of normal (<27 or >31)')

1.25% of the data reports an atmospheric pressure outside of normal (<27 or >31)


In [39]:
df_clean = df_clean[(df_clean["Pressure(in)"] >= 25) & (df_clean["Pressure(in)"] <= 31)]

In [40]:
print(f'{100 * (df_clean["Visibility(mi)"] > 20).sum() / df_clean.shape[0]:.5f}% of accident occurred with a visibility of 20 miles or more')

0.10369% of accident occurred with a visibility of 20 miles or more


In [41]:
df_clean = df_clean[df_clean["Visibility(mi)"] <= 20]

In [42]:
print(f'{100 * (df_clean["Wind_Speed(mph)"] > 150).sum() / df_clean.shape[0]:.5f}% of the data reports a wind speeds of over 150mph')

0.00063% of the data reports a wind speeds of over 150mph


In [43]:
df_clean = df_clean[df_clean["Wind_Speed(mph)"] <= 150]

In [44]:
print(f"{100 * (init_nrows - df_clean.shape[0]) / init_nrows:.2f}% of the original dataset was dropped")

5.18% of the original dataset was dropped


#### Boolean Variables

No aparent incosistencies as all were automatically detected as `bool`.

#### Categorical Variables

In [45]:
df_clean["City"].unique()

array(['Dayton', 'Reynoldsburg', 'Williamsburg', ..., 'Pond Eddy',
       'Clarksdale', 'American Fork-Pleasant Grove'], dtype=object)

In [46]:
df_clean["County"].unique()

array(['Montgomery', 'Franklin', 'Clermont', ..., 'Mcminn', 'Glascock',
       'Woods'], dtype=object)

Only data from the State of Alaska is missing.

In [47]:
df_clean["State"].unique().shape

(49,)

In [48]:
df_clean["State"].unique()

array(['OH', 'WV', 'CA', 'FL', 'GA', 'SC', 'NE', 'IA', 'IL', 'MO', 'WI',
       'IN', 'MI', 'NJ', 'NY', 'CT', 'MA', 'RI', 'NH', 'PA', 'KY', 'MD',
       'VA', 'DC', 'DE', 'TX', 'WA', 'OR', 'AL', 'NC', 'AZ', 'TN', 'LA',
       'MN', 'OK', 'NV', 'UT', 'KS', 'NM', 'AR', 'CO', 'MS', 'ME', 'VT',
       'ID', 'ND', 'MT', 'SD', 'WY'], dtype=object)

### 2.3. Extract More Data and Rename Columns

We can extract the road type from the `Street` name.

In [49]:
def road_type(street_name):
    # Check if the street name contains common highway prefixes or suffixes
    highway_prefixes = ['I-', 'US-', 'SR-', 'HWY', 'INTERSTATE', 'US HIGHWAY', 'STATE ROUTE']
    highway_suffixes = ['INTERSTATE', 'HIGHWAY', 'EXPRESSWAY', 'TURNPIKE', 'PARKWAY', 'ROUTE']
    
    for prefix in highway_prefixes:
        if street_name.upper().startswith(prefix):
            return 'Highway'
    
    for suffix in highway_suffixes:
        if street_name.upper().endswith(suffix):
            return 'Highway'
    
    return 'Local Road'

In [50]:
df_clean["Road_Type"] = df_clean["Street"].apply(road_type)

In [51]:
df_clean["Road_Type"].value_counts()

Local Road    5296691
Highway       2031031
Name: Road_Type, dtype: int64

Rename column

In [52]:
df_clean = df_clean.rename(columns={"Sunrise_Sunset": "Day/Night"})

### 2.4. Save Clean Data

In [53]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7327722 entries, 0 to 7728393
Data columns (total 36 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 object 
 1   Severity           int64  
 2   Start_Time         object 
 3   End_Time           object 
 4   Start_Lat          float64
 5   Start_Lng          float64
 6   Distance(mi)       float64
 7   Street             object 
 8   City               object 
 9   County             object 
 10  State              object 
 11  Temperature(F)     float64
 12  Humidity(%)        float64
 13  Pressure(in)       float64
 14  Visibility(mi)     float64
 15  Wind_Speed(mph)    float64
 16  Precipitation(in)  float64
 17  Weather_Condition  object 
 18  Amenity            bool   
 19  Bump               bool   
 20  Crossing           bool   
 21  Give_Way           bool   
 22  Junction           bool   
 23  No_Exit            bool   
 24  Railway            bool   
 25  Roundabout        

In [54]:
#df_clean.to_csv("data/US_Accidents_March23_Clean.csv", index=False)